# 2. Pre-Processing & Baseline modeling

## 2.1 Contents <a id='2.1_Contents'></a>
* [2.1 Contents](#2.1_Contents)
* [2.2 Introduction](#2.2)
* [2.3 Imports](#2.3)
* [2.4 Loading Data](#2.4)
* [2.5 Set Options](#2.5)

## 2.2 Introduction <a id='2.2'></a>

## 2.3 Imports <a id='2.3'></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf
import pandasql as psql
import sqlite3

# stats packages
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_squared_error

# modeling packages
from sklearn.model_selection import train_test_split
import xgboost as xgb





## 2.4 Loading Data <a id='2.4'></a>

In [2]:
campaign_desc = pd.read_csv(r"C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\Capstone 3\exports\campaign_desc.csv")
campaign_table = pd.read_csv(r"C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\Capstone 3\exports\campaign_table.csv")
coupon = pd.read_csv(r"C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\Capstone 3\exports\coupon.csv")
coupon_redempt = pd.read_csv(r"C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\Capstone 3\exports\coupon_redempt.csv")
hh_demographic = pd.read_csv(r"C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\Capstone 3\exports\hh_demographic.csv")
product = pd.read_csv(r"C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\Capstone 3\exports\product.csv")
transaction_data = pd.read_csv(r"C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\Capstone 3\exports\transaction_data.csv")
demo_coupon = pd.read_csv(r"C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\Capstone 3\exports\demo_coupon")
demo_no_disc = pd.read_csv(r"C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\Capstone 3\exports\demo_no_disc")
merged_data = pd.read_csv(r"C:\Users\leann\OneDrive\Desktop\SPRINGBOARD\Capstone 3\exports\merged_data")


## 2.5 Set Options <a id='2.5'></a>

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# Set up my psql
pysql = lambda q: sqldf(q, globals())

## 2.6 Pre-Processing <a id='2.6'></a>

### 2.6.1 Change Data Types <a id='1.6.1'></a>

In [4]:
# check datatypes to make sure that nothing needs to be fixed.

print(campaign_desc.dtypes)

campaign_desc['description']=campaign_desc['description'].astype('category')
campaign_desc['campaign']=campaign_desc['campaign'].astype('category')

description      object
campaign          int64
start_day         int64
end_day           int64
duration_days     int64
dtype: object


In [5]:
print(campaign_table.dtypes)
campaign_table['campaign']=campaign_table['campaign'].astype('category')
campaign_table['description']=campaign_table['description'].astype('category')

description      object
household_key     int64
campaign          int64
dtype: object


In [6]:
print(coupon.dtypes)
coupon['campaign']=coupon['campaign'].astype('category')

coupon_upc    int64
product_id    int64
campaign      int64
dtype: object


In [7]:
print(coupon_redempt.dtypes)

coupon_redempt['campaign']=coupon_redempt['campaign'].astype('category')

household_key    int64
day              int64
coupon_upc       int64
campaign         int64
dtype: object


In [8]:
print(hh_demographic.dtypes)
cols_to_convert = {
    'age_desc': 'category',
    'marital_status': 'category',
    'income_desc': 'category',
    'homeowner_desc': 'category',
    'hh_comp_desc': 'category',
    'household_size_desc': 'category',
    'kid_category_desc': 'category'
}

hh_demographic = hh_demographic.astype(cols_to_convert)


age_desc               object
marital_status         object
income_desc            object
homeowner_desc         object
hh_comp_desc           object
household_size_desc    object
kid_category_desc      object
household_key           int64
dtype: object


In [9]:
print(product.dtypes)
cols_to_convert  = {
    'department': 'category',
    'brand': 'category',
    'commodity_desc':'category',
    'sub_commodity_desc': 'category'
}

product = product.astype(cols_to_convert)

product_id             int64
manufacturer           int64
department            object
brand                 object
commodity_desc        object
sub_commodity_desc    object
dtype: object


In [10]:
print(transaction_data.dtypes)

transaction_data['store_id'] = transaction_data['store_id'].astype('category')

# NOTE - I'm not going to worry about converting the trans_time to datetime as I'm not going to use that metric anymore.

household_key          int64
basket_id              int64
store_id               int64
day                    int64
week_no                int64
trans_time            object
product_id             int64
sales_value          float64
amt_paid             float64
retail_disc          float64
coupon_disc          float64
coupon_match_disc    float64
day_of_week            int64
hour                   int64
dtype: object


In [11]:
demo_coupon.head()


,age_desc,marital_status,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,household_key,basket_id,store_id,day,week_no,trans_time,product_id,sales_value,amt_paid,retail_disc,coupon_disc,coupon_match_disc,coupon_upc,campaign
0,45-54,Married,25-34K,Homeowner,2 Adults Kids,5+,3+,718,26985360571,324,1,1,11:15,830503,2.99,0.99,-1.0,-1.0,0.0,51111070124,30
1,45-54,Married,25-34K,Homeowner,2 Adults Kids,5+,3+,718,26985360571,324,1,1,11:15,830503,2.99,0.99,-1.0,-1.0,0.0,51111074242,26
2,45-54,Married,25-34K,Homeowner,2 Adults Kids,5+,3+,718,26985360571,324,1,1,11:15,830503,2.99,0.99,-1.0,-1.0,0.0,53600000082,10
3,45-54,Married,25-34K,Homeowner,2 Adults Kids,5+,3+,718,26985360571,324,1,1,11:15,830503,2.99,0.99,-1.0,-1.0,0.0,53620010028,27
4,45-54,Married,25-34K,Homeowner,2 Adults Kids,5+,3+,718,26985360571,324,1,1,11:15,830503,2.99,0.99,-1.0,-1.0,0.0,55703711082,18


In [12]:
print(demo_coupon.dtypes)

cols_to_convert = {
    'age_desc': 'category',
    'marital_status': 'category',
    'income_desc': 'category',
    'homeowner_desc': 'category',
    'hh_comp_desc': 'category',
    'household_size_desc': 'category',
    'kid_category_desc': 'category',
    'store_id': 'category',
    'campaign': 'category'   
}

demo_coupon = demo_coupon.astype(cols_to_convert)

age_desc                object
marital_status          object
income_desc             object
homeowner_desc          object
hh_comp_desc            object
household_size_desc     object
kid_category_desc       object
household_key            int64
basket_id                int64
store_id                 int64
day                      int64
week_no                  int64
trans_time              object
product_id               int64
sales_value            float64
amt_paid               float64
retail_disc            float64
coupon_disc            float64
coupon_match_disc      float64
coupon_upc               int64
campaign                 int64
dtype: object


In [13]:
print(demo_no_disc.dtypes)
cols_to_convert = {
    'age_desc': 'category',
    'marital_status': 'category',
    'income_desc': 'category',
    'homeowner_desc': 'category',
    'hh_comp_desc': 'category',
    'household_size_desc': 'category',
    'kid_category_desc': 'category',
    'store_id': 'category'   
}

demo_no_disc = demo_no_disc.astype(cols_to_convert)

age_desc                object
marital_status          object
income_desc             object
homeowner_desc          object
hh_comp_desc            object
household_size_desc     object
kid_category_desc       object
household_key            int64
basket_id                int64
store_id                 int64
store_id.1               int64
day                      int64
week_no                  int64
trans_time              object
product_id               int64
sales_value            float64
amt_paid               float64
retail_disc            float64
coupon_disc            float64
coupon_match_disc      float64
dtype: object


In [14]:
merged_data.dtypes

cols_to_convert = {
    'age_desc': 'category',
    'marital_status': 'category',
    'income_desc': 'category',
    'homeowner_desc': 'category',
    'hh_comp_desc': 'category',
    'household_size_desc': 'category',
    'kid_category_desc': 'category',
    'commodity_desc': 'category'
}

# Rename to something more descriptive
demo_sales_by_commodity = merged_data.astype(cols_to_convert)

## Build Table for Modeling
Target: sales_value
Features: day, household_key(?), basket_id(?), product_id(?)coupon_disc, retail_disc, coupon_match_disc, amt_paid, demographics

In [15]:
transaction_data

,household_key,basket_id,store_id,day,week_no,trans_time,product_id,sales_value,amt_paid,retail_disc,coupon_disc,coupon_match_disc,day_of_week,hour
0,2375,26984851472,364,1,1,16:31:00,1004906,1.39,0.79,-0.60,0.0,0.0,1,16
1,2375,26984851472,364,1,1,16:31:00,1033142,0.82,0.82,0.00,0.0,0.0,1,16
2,2375,26984851472,364,1,1,16:31:00,1036325,0.99,0.69,-0.30,0.0,0.0,1,16
3,2375,26984851472,364,1,1,16:31:00,1082185,1.21,1.21,0.00,0.0,0.0,1,16
4,2375,26984851472,364,1,1,16:31:00,8160430,1.50,1.11,-0.39,0.0,0.0,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2581261,1598,42305362535,3228,711,102,15:20:00,92130,0.99,0.99,0.00,0.0,0.0,4,15
2581262,1598,42305362535,3228,711,102,15:20:00,114102,8.89,8.89,0.00,0.0,0.0,4,15
2581263,1598,42305362535,3228,711,102,15:20:00,133449,6.99,6.99,0.00,0.0,0.0,4,15
2581264,1598,42305362535,3228,711,102,15:20:00,6923644,4.50,4.01,-0.49,0.0,0.0,4,15


In [16]:
# First select the columns from transaction_data that I will use, according to AJ
query = """
SELECT household_key, day, sales_value, retail_disc, coupon_disc, coupon_match_disc
FROM transaction_data
ORDER BY day;
"""

transaction_data_selected = psql.sqldf(query, locals())
transaction_data_selected

,household_key,day,sales_value,retail_disc,coupon_disc,coupon_match_disc
0,2375,1,1.39,-0.60,0.0,0.0
1,2375,1,0.82,0.00,0.0,0.0
2,2375,1,0.99,-0.30,0.0,0.0
3,2375,1,1.21,0.00,0.0,0.0
4,2375,1,1.50,-0.39,0.0,0.0
...,...,...,...,...,...,...
2581261,1598,711,0.99,0.00,0.0,0.0
2581262,1598,711,8.89,0.00,0.0,0.0
2581263,1598,711,6.99,0.00,0.0,0.0
2581264,1598,711,4.50,-0.49,0.0,0.0


In [17]:
transaction_data_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2581266 entries, 0 to 2581265
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   household_key      int64  
 1   day                int64  
 2   sales_value        float64
 3   retail_disc        float64
 4   coupon_disc        float64
 5   coupon_match_disc  float64
dtypes: float64(4), int64(2)
memory usage: 118.2 MB


In [18]:
query = """
SELECT t.*, h.age_desc AS age, h.marital_status, h.income_desc AS income, h.homeowner_desc AS homeowner, h.hh_comp_desc AS hh_comp, h.household_size_desc AS hh_size, h.kid_category_desc AS kids
FROM transaction_data_selected as t
INNER JOIN hh_demographic as h
ON t.household_key = h.household_key
ORDER BY day;
"""

transaction_demo = psql.sqldf(query, locals())

In [19]:
transaction_demo.tail()

,household_key,day,sales_value,retail_disc,coupon_disc,coupon_match_disc,age,marital_status,income,homeowner,hh_comp,hh_size,kids
1419126,1369,711,1.29,0.0,0.0,0.0,25-34,Single,Under 15K,Probable Renter,Single Male,1,None/Unknown
1419127,1369,711,1.69,-0.4,0.0,0.0,25-34,Single,Under 15K,Probable Renter,Single Male,1,None/Unknown
1419128,1369,711,25.98,0.0,0.0,0.0,25-34,Single,Under 15K,Probable Renter,Single Male,1,None/Unknown
1419129,1369,711,1.99,0.0,0.0,0.0,25-34,Single,Under 15K,Probable Renter,Single Male,1,None/Unknown
1419130,1369,711,2.69,-0.7,0.0,0.0,25-34,Single,Under 15K,Probable Renter,Single Male,1,None/Unknown


In [20]:
# Change the discount types to binary so we don't get leakage

# Define columns to be converted to binary format
cols_to_binary = ['retail_disc', 'coupon_disc', 'coupon_match_disc']

# Convert columns/change type to int8 to be more memory efficient
transaction_demo[cols_to_binary] = transaction_demo[cols_to_binary].applymap(lambda x: 1 if x < 0 else 0).astype('int8')


In [21]:
transaction_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1419131 entries, 0 to 1419130
Data columns (total 13 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   household_key      1419131 non-null  int64  
 1   day                1419131 non-null  int64  
 2   sales_value        1419131 non-null  float64
 3   retail_disc        1419131 non-null  int8   
 4   coupon_disc        1419131 non-null  int8   
 5   coupon_match_disc  1419131 non-null  int8   
 6   age                1419131 non-null  object 
 7   marital_status     1419131 non-null  object 
 8   income             1419131 non-null  object 
 9   homeowner          1419131 non-null  object 
 10  hh_comp            1419131 non-null  object 
 11  hh_size            1419131 non-null  object 
 12  kids               1419131 non-null  object 
dtypes: float64(1), int64(2), int8(3), object(7)
memory usage: 112.3+ MB


In [22]:
# coupon names
transaction_demo.columns

Index(['household_key', 'day', 'sales_value', 'retail_disc', 'coupon_disc',
       'coupon_match_disc', 'age', 'marital_status', 'income', 'homeowner',
       'hh_comp', 'hh_size', 'kids'],
      dtype='object')

In [23]:
# Convert objects to categorical

cols_to_convert = {
    'age': 'category', 
    'marital_status': 'category', 
    'income': 'category',
    'homeowner': 'category', 
    'hh_comp': 'category', 
    'hh_size': 'category',
    'kids': 'category'
    
}

for column, dtype in cols_to_convert.items():
    if column in transaction_demo.columns:
        transaction_demo[column] = transaction_demo[column].astype(dtype)

transaction_demo.dtypes

household_key           int64
day                     int64
sales_value           float64
retail_disc              int8
coupon_disc              int8
coupon_match_disc        int8
age                  category
marital_status       category
income               category
homeowner            category
hh_comp              category
hh_size              category
kids                 category
dtype: object

In [25]:
transaction_demo

,household_key,day,sales_value,retail_disc,coupon_disc,coupon_match_disc,age,marital_status,income,homeowner,hh_comp,hh_size,kids
0,1364,1,2.19,0,0,0,65+,Single,100-124K,Homeowner,Single Female,1,None/Unknown
1,1364,1,2.99,1,0,0,65+,Single,100-124K,Homeowner,Single Female,1,None/Unknown
2,1364,1,3.09,0,0,0,65+,Single,100-124K,Homeowner,Single Female,1,None/Unknown
3,1364,1,2.50,1,0,0,65+,Single,100-124K,Homeowner,Single Female,1,None/Unknown
4,1364,1,0.60,1,0,0,65+,Single,100-124K,Homeowner,Single Female,1,None/Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419126,1369,711,1.29,0,0,0,25-34,Single,Under 15K,Probable Renter,Single Male,1,None/Unknown
1419127,1369,711,1.69,1,0,0,25-34,Single,Under 15K,Probable Renter,Single Male,1,None/Unknown
1419128,1369,711,25.98,0,0,0,25-34,Single,Under 15K,Probable Renter,Single Male,1,None/Unknown
1419129,1369,711,1.99,0,0,0,25-34,Single,Under 15K,Probable Renter,Single Male,1,None/Unknown


Perform one hot encoding on the categorical features:

In [26]:
# use the dictionary I already made of the columns to convert to categorical
cols_to_encode = list(cols_to_convert.keys())


transaction_demo_encoded = pd.get_dummies(transaction_demo, columns=cols_to_encode)


In [27]:
# rename to make it easier to type moving forward

data = transaction_demo_encoded 

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1419131 entries, 0 to 1419130
Data columns (total 47 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   household_key              1419131 non-null  int64  
 1   day                        1419131 non-null  int64  
 2   sales_value                1419131 non-null  float64
 3   retail_disc                1419131 non-null  int8   
 4   coupon_disc                1419131 non-null  int8   
 5   coupon_match_disc          1419131 non-null  int8   
 6   age_19-24                  1419131 non-null  uint8  
 7   age_25-34                  1419131 non-null  uint8  
 8   age_35-44                  1419131 non-null  uint8  
 9   age_45-54                  1419131 non-null  uint8  
 10  age_55-64                  1419131 non-null  uint8  
 11  age_65+                    1419131 non-null  uint8  
 12  marital_status_Married     1419131 non-null  uint8  
 13  marital_stat

<b>Train Test Split:</b>

In [29]:
# Make sure that our data is sorted by day
data = data.sort_values(by='day')

# Divide data into Y and y
X = data.drop('sales_value', axis=1)
y = data['sales_value']

# I don't want the train/test splits to get the days mixed up, I want to keep the days together, in order.
# I also don't want a split to happen in the middle of a day.

# Identify day that is at the 80% mark
unique_days = transaction_demo_encoded['day'].unique()
threshold_day = unique_days[int(0.8 * len(unique_days))]
print('threshold day: ', threshold_day)

threshold day:  569


In [31]:
# Figure out what data goes in train/test based on threshold day
train_indices = data[data['day'] < threshold_day].index
test_indices = data[data['day'] >= threshold_day].index



In [32]:
# Train test split

X_train, X_test, y_train, y_test = X.iloc[train_indices], X.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]

## Build baseline model: XG Boost

In [33]:
# Build the model
xg_model = xgb.XGBRegressor(objective='reg:squarederror')

# Fit
xg_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [34]:
# Predict
y_pred_xg = xg_model.predict(X_test)

In [35]:
mse = mean_squared_error(y_test, y_pred_xg)
rmse = np.sqrt(mse)

print("RMSE:", rmse)

# Average sales_value:
av_sale= np.average(data['sales_value'])
print("Average Sale: ", av_sale)
print("% error: ", rmse/av_sale * 100,"%")

RMSE: 4.540190484982664
Average Sale:  3.169342090335567
% error:  143.25340577236182 %


In [36]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

mape = mean_absolute_percentage_error(y_test, y_pred_xg)
print(f"MAPE: {mape:.2f}%")


MAPE: 138.97%


<b> We have high error! </b>